In [12]:
!pip install -q pydantic openai rich

In [13]:
# import all the required libraries
import getpass
from pydantic import BaseModel
from openai import OpenAI
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
import time

In [14]:
# --- Configuration ---
API_KEY = None
MODEL = "gemini-2.0-flash"
SYSTEM_PROMPT = """You are an intelligent agent capable of understanding and responding to user queries.
                You can access tools to perform actions and provide information.
                If you need to use a tool, format your response as: {{tool:tool_name, input:tool_input}}.
                If you have a final answer, provide it directly.
                """

In [15]:
# --- Tools (Example) ---
def search_web(query: str) -> str:
    """Simulated web search tool."""
    print(f"Searching the web for: {query}")
    time.sleep(1)  # Simulate delay
    return f"Web search results for '{query}': ... (simulated results)"

def calculate(expression: str) -> str:
    """Simulated calculator tool."""
    print(f"Calculating: {expression}")
    time.sleep(1) # simulate delay
    try:
        result = eval(expression) # be very careful with eval in production
        return str(result)
    except:
        return "Invalid Calculation"

TOOLS = {
    "search_web": search_web,
    "calculate": calculate,
}

In [16]:
# --- Pydantic Models ---
class UserMessage(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    model: str
    messages: list[UserMessage]
    n: int = 1

class ChatResponse(BaseModel):
    response: str

In [9]:
# --- OpenAI Client ---
def initialize_client():
    global API_KEY, client
    try:
        API_KEY = getpass.getpass(prompt="Enter your API key: ")
        print("API Key added.")
        client = OpenAI(
            api_key=API_KEY,
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
        )
    except Exception as e:
        print(f"Error initializing client: {e}")
        exit(1)

In [10]:
# --- Agent Functionality ---
def get_ai_response(messages: list[UserMessage]) -> ChatResponse:
    """Get AI response."""
    chat_request = ChatRequest(
        model=MODEL,
        messages=messages,
    )
    response = client.chat.completions.create(
        model=chat_request.model,
        n=chat_request.n,
        messages=[message.model_dump() for message in chat_request.messages]
    )
    return ChatResponse(response=response.choices[0].message.content)

In [17]:
# --- Main Function For Running the Agent ---
def run_agent():
    """Main agent loop."""
    global client
    initialize_client()
    console = Console()
    messages = [UserMessage(role="system", content=SYSTEM_PROMPT)]

    while True:
        user_input = input("User: ")
        if user_input.lower() == "quit":
            print("Goodbye!")
            break

        messages.append(UserMessage(role="user", content=user_input))
        ai_response = get_ai_response(messages)
        ai_response_text = ai_response.response

        console.print(Panel(user_input, title="[bold blue]User Question[/bold blue]", border_style="bright_blue"))
        console.print(Panel(Markdown(ai_response_text), title="[bold green]AI Response[/bold green]", border_style="bright_green"))

        # Tool Parsing and Execution
        if "{{tool:" in ai_response_text:
            try:
                start = ai_response_text.find("{{tool:") + 7
                end = ai_response_text.find("}}", start)
                tool_call = ai_response_text[start:end]
                tool_name, tool_input = tool_call.split(", input:")
                tool_result = TOOLS[tool_name](tool_input)

                # add the assistants response to the messages
                messages.append(UserMessage(role="assistant", content=ai_response_text))

                # add the tool result to the messages
                messages.append(UserMessage(role="tool", content=f"tool_result: {tool_result}"))

                # get the response with the tool results integrated.
                ai_response = get_ai_response(messages)

                console.print(Panel(Markdown(ai_response.response), title="[bold yellow]Tool Integrated Response[/bold yellow]", border_style="bright_yellow"))
            except Exception as e:
                console.print(f"Error executing tool: {e}")

        else:
             # add the assistants response to the messages
            messages.append(UserMessage(role="assistant", content=ai_response_text))
            pass

In [18]:
# test the agent
run_agent()

Enter your API key: ··········
API Key added.
User: what is love?


╭───────────────────────────────────────────────── User Question ─────────────────────────────────────────────────╮
│ what is love?                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── AI Response ──────────────────────────────────────────────────╮
│ Love is a complex emotion with a wide range of meanings and interpretations, often varying from person to       │
│ person and culture to culture. It can encompass feelings of affection, care, warmth, and attachment. Here are   │
│ some common aspects of love:                                                                                    │
│                                                                                                                 │
│  • Affection: A feeling of warmth and fondness towards someone or something.                                    │
│  • Intimacy: A sense of closeness, understanding, and connection with another person.                           │
│  • Passion: Strong feelings of excitement, desire, and attraction.                                              │
│  • Commitment: A conscious decision to stay with someone through good times and bad.                            │
│  • Compassion: Empathy and concern for the well-being of others.                                                │
│  • Selflessness: Prioritizing the needs and happiness of others.                                                │
│                                                                                                                 │
│ Love can also manifest in different forms, such as:                                                             │
│                                                                                                                 │
│  • Romantic Love: Characterized by passion, intimacy, and commitment.                                           │
│  • Familial Love: The bond between family members, often based on shared history and unconditional acceptance.  │
│  • Platonic Love: A deep friendship without romantic or sexual feelings.                                        │
│  • Self-Love: Acceptance, respect, and appreciation for oneself.                                                │
│  • Agape: Unconditional, selfless love for all beings.                                                          │
│                                                                                                                 │
│ Ultimately, love is a powerful and multifaceted emotion that enriches our lives and relationships.              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

User: quit
Goodbye!
